In [2]:
import heapq, random

class PriorityQueue:
    """
      Implements a priority queue data structure. Each inserted item
      has a priority associated with it and the client is usually interested
      in quick retrieval of the lowest-priority item in the queue. This
      data structure allows O(1) access to the lowest-priority item.
    """
    def  __init__(self):
        self.heap = []
        self.count = 0

    def push(self, item, priority):
        entry = (priority, self.count, item)
        heapq.heappush(self.heap, entry)
        self.count += 1

    def pop(self):
        (_, _, item) = heapq.heappop(self.heap)
        return item

    def isEmpty(self):
        return len(self.heap) == 0

    def update(self, item, priority):
        # If item already in priority queue with higher priority, update its priority and rebuild the heap.
        # If item already in priority queue with equal or lower priority, do nothing.
        # If item not in priority queue, do the same thing as self.push.
        for index, (p, c, i) in enumerate(self.heap):
            if i == item:
                if p <= priority:
                    break
                del self.heap[index]
                self.heap.append((priority, c, item))
                heapq.heapify(self.heap)
                break
        else:
            self.push(item, priority)

In [3]:
class Person:
    def __init__(self, height: int) -> None:
        """
        height: height in inches rounded to nearest whole number
        """
        self.height = height
        self.wingspan = height * 1.06
        self.reach = height * 1.35
        self.leg_length = height * 0.5

In [4]:
class Hold:
    def __init__(self, x: int, y: int, diff: float, width: float, height: float, angle: int):
        #Coords = top left corner
        self.x = x
        self.y = y
        self.diff = diff
        self.width = width
        self.height = height
        self.angle = angle
    def getCenter(self):
        return (self.coords[0] + self.width/2, self.coords[1] + self.height/2)
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    def __gt__(self, other):
        return self.y > other.y
    def __lt__(self, other):
        return self.y < other.y
    def __ge__(self, other):
        return self.y >= other.y
    def __le__(self, other):
        return self.y <= other.y
    
    def __repr__(self):
        return (f"""Hold: Top left at {self.x}, {self.y}
                Width = {self.width}, Height = {self.height}
                Difficulty = {round(self.diff, 2)}/10, Angle = {self.angle} degrees\n""")

In [5]:
from enum import Enum

class LimbName(Enum):
    LEFT_HAND = "Left Hand"
    RIGHT_HAND = "Right Hand"
    LEFT_LEG = "Left Leg"
    RIGHT_LEG = "Right Leg"


In [6]:
class Limb:
    def __init__(self, name: LimbName, strength: int, flexibility: int, hold: Hold):
        self.name, self.strength, self.flexibility, self.hold = name, strength, flexibility, hold
    def __repr__(self):
        return (f"{self.name} at {self.hold}")

In [7]:
class Route:
    def __init__(self, holds: Hold, start1: Hold, start2: Hold, finish: Hold):
        self.holds = holds
        self.start_hold1 = start1
        self.start_hold2 = start2
        self.finish_hold = finish

In [8]:
import math
from copy import copy

class State:
    def __init__(self, lf: Limb, rf: Limb, lh: Limb, rh: Limb, person: Person, route: Route):
        self.lf, self.rf, self.lh, self.rh = lf, rf, lh, rh
        self.person = person
        self.moves = []
        self.costs = []
        self.route = route
        self.wall_height_inches = 180 # Represents real height of wall <- need to measure
        self.wall_height_pixels = 3321 #placeholder <- Need to extract from image

    # Not sure where to put this
    def inches_to_pixels(self, inches: int):
        return (self.wall_height_pixels / self.wall_height_inches) * inches

    def __eq__(self, other):
        return ((self.lf.hold == other.lf.hold and self.rf.hold == other.rf.hold 
                and self.rh.hold == other.rh.hold and self.lh.hold == other.lh.hold)
                or (self.lf.hold == other.rf.hold and self.rf.hold == other.lf.hold 
                and self.rh.hold == other.lh.hold and self.lh.hold == other.rh.hold))

    def __repr__(self):
        return f"{self.moves}"

    def getStateSuccessors(self):
        succs = []
        limbs = [self.lf, self.rf, self.lh, self.rh]
        for i in range(len(limbs)):
            neighs = self.getNeighbors(limbs[i])
            for neigh in neighs:
                #print("Hi neighbor")
                new_state = copy(self)
                if i == 0:
                    new_state.lf = Limb(LimbName.LEFT_LEG, 2.5, 8, neigh)
                    action = (state.lf, neigh)
                if i == 1:
                    new_state.rf = Limb(LimbName.RIGHT_LEG, 2.5, 8, neigh)
                    action = (state.rf, neigh)
                if i == 2:
                    new_state.lh = Limb(LimbName.LEFT_HAND, 8, 2, neigh)
                    action = (state.lh, neigh)
                if i == 3:
                    new_state.rh = Limb(LimbName.RIGHT_HAND, 8, 2, neigh)
                    action = (self.rh, neigh)
                succs.append((new_state, action))
        #print(succs)
        return succs

    def getNeighbors(self, limb):
        neighbors = []
        for hold in self.route.holds:
            if not hold == limb.hold:
                if limb.name in [LimbName.LEFT_LEG, LimbName.RIGHT_LEG]:
                    #print("Checking leg neighbors")
                    #print(f"Height diff: {abs(hold.y - limb.hold.y)}")
                    #print(f"Leg length: {self.inches_to_pixels(self.person.leg_length)}")
                    low_arm = max([self.lh.hold.y, self.rh.hold.y])
                    if (0 < limb.hold.y - hold.y < self.inches_to_pixels(self.person.leg_length) and 
                        abs(hold.x - limb.hold.x) < self.inches_to_pixels(self.person.leg_length) and
                        hold.y > low_arm): 
                        neighbors.append(hold)
                elif limb.name in [LimbName.LEFT_HAND, LimbName.RIGHT_HAND]:
                    #print("Checking arm neighbors")
                    #print("HAND TIME 1")
                    upper_leg, lower_leg = sorted([self.lf.hold.y, self.rf.hold.y])
                    if (abs(hold.x - limb.hold.x) < self.inches_to_pixels(self.person.wingspan) and
                        lower_leg - hold.y < self.inches_to_pixels(self.person.height * 0.8) and hold.y < upper_leg and 0 < limb.hold.y - hold.y):
                        #print(hold.y - upper_leg)
                        #print("HAND TIME 2")
                        neighbors.append(hold)
        return neighbors
        
def moveDifficulty(state: State, limb: Limb, next_hold: Hold):
    distance = math.sqrt(((limb.hold.x - next_hold.x) ** 2) + ((limb.hold.y - next_hold.y) ** 2))
    distance_diff = distance

    new_state = State(copy(state.lf), copy(state.rf), copy(state.lh), copy(state.rh), state.person, state.route)
    new_state_limbs = [new_state.lh, new_state.rh, new_state.lf, new_state.rf]
    for new_state_limb in new_state_limbs:
        if new_state_limb.name == limb.name:
            new_state_limb.hold = None
    state_without_limb_difficulty = 0.3 * stateDifficulty(new_state)
    distance_diff *= 0.05
    move_diff = distance_diff + state_without_limb_difficulty
    return move_diff

def stateDifficulty(state: State):
    if state.lh.hold != None and state.rh.hold != None:
        average_hands_x = (state.lh.hold.x + state.rh.hold.x) / 2
        average_hands_y = (state.lh.hold.y + state.rh.hold.y) / 2
    else:
        if state.rh.hold == None:
            average_hands_x = state.lh.hold.x  
            average_hands_y = state.lh.hold.y
        else:
            average_hands_y = state.rh.hold.y
            average_hands_x = state.rh.hold.x
    if state.lf.hold != None and state.rf.hold != None:
        average_legs_x = (state.lf.hold.x + state.rf.hold.x) / 2
        average_legs_y = (state.lf.hold.y + state.rf.hold.y) / 2 
    else:
        if state.rf.hold == None:
            average_legs_x = state.lf.hold.x
            average_legs_y = state.lf.hold.y
        else:
            average_legs_y = state.rf.hold.y
            average_legs_x = state.rf.hold.x

    hands_difference_x = abs(state.lh.hold.x - state.rh.hold.x) if state.lh.hold != None and state.rh.hold != None else 0
    hands_difference_y = abs(state.lh.hold.y - state.rh.hold.y) if state.lh.hold != None and state.rh.hold != None else 0

    legs_difference_x = abs(state.lf.hold.x - state.rf.hold.x) if state.lf.hold != None and state.rf.hold != None else 0
    legs_difference_y = abs(state.lf.hold.y - state.rf.hold.y) if state.lf.hold != None and state.rf.hold != None else 0

    diff = 0
    center_diff = 0.5*abs(average_hands_x - average_legs_x)

    target_distance = state.inches_to_pixels(state.inches_to_pixels(state.person.height * 0.8))
    scaling_factor = 1
    distance_diff = abs(average_legs_y - average_hands_y) - target_distance

    raw_difficulty_score = (scaling_factor * (distance_diff ** 2))

    # Scales the difficulty score linearly between 1 and 10

    limb_strength_diff = 0
    angle_diff = 0
    for limb in [state.lh, state.rh, state.lf, state.rf]:
        if limb.hold != None:
            limb_strength_diff += limb.hold.diff / limb.strength
            if limb.name == LimbName.LEFT_HAND:
                if 315 >= limb.hold.angle >= 270:
                    angle_diff += 2
                elif 90 >= limb.hold.angle or limb.hold.angle > 315:
                    angle_diff += 1
                elif 180 >= limb.hold.angle > 90:
                    angle_diff += 2.5
                else:
                    angle_diff += 3
            if limb.name == LimbName.RIGHT_HAND:
                if 90 >= limb.hold.angle >= 45:
                    angle_diff += 2
                elif 45 >= limb.hold.angle or limb.hold.angle > 270:
                    angle_diff += 1
                elif 270 >= limb.hold.angle > 180:
                    angle_diff += 2.5
                else:
                    angle_diff += 3
            if limb.name in [LimbName.LEFT_LEG, LimbName.RIGHT_LEG]:
                if 90 <= limb.hold.angle <= 270:
                    angle_diff += 2
        else:
            limb_strength_diff += 6
    separation_diff = 0
    separation_diff += hands_difference_y 
    if hands_difference_x > state.inches_to_pixels(state.inches_to_pixels(0.8 * state.person.wingspan)):
        separation_diff += 0.5 * hands_difference_x
    separation_diff += 0.5 * legs_difference_y
    if legs_difference_x > state.inches_to_pixels(state.inches_to_pixels(0.6 * state.person.wingspan)):
        separation_diff += 0.5 * legs_difference_x
    center_diff *= 0.01
    raw_difficulty_score *= 0.00000002
    angle_diff *= 1
    limb_strength_diff *= 2
    separation_diff *= 0.05
    #print(f"{separation_diff}")
    diff += center_diff + raw_difficulty_score + angle_diff + limb_strength_diff + separation_diff
    #print(f"center = {center_diff}, scaled = {raw_difficulty_score}, angle = {angle_diff}, strength = {limb_strength_diff}")
    return diff

In [9]:
import numpy as np
from copy import copy
class aStar:
    WALL_HEIGHT = 180 #image.height -> divide that by height of wall in inches to get conversion
    def __init__(self, state):
        self.state = state
        self.reach = state.person.reach

    
                    
    def get_distance(self, hold, state):
        return np.sqrt((state[0] - hold[0]) ** 2 + (state[1] - hold[1]) ** 2)

    def getCostValue(self, costs):
        return sum([cost ** 2 for cost in costs])
        
    def uniformCostSearch(self):
        explored = []
        frontier = PriorityQueue()
        num = 0
        frontier.push(self.state, 0)
        while not frontier.isEmpty():
            cur_state = frontier.pop()
            #print(cur_state.moves)
            #print(f"Looking at state {cur_state}")
            explored.append(cur_state)
            if cur_state.lh.hold == self.state.route.finish_hold or cur_state.rh.hold == self.state.route.finish_hold:
                print("TAAAAAAAAAAAAAAKE")
                print(cur_state.costs)
                print(self.getCostValue(cur_state.costs))
                return cur_state.moves
            for next_state, action in cur_state.getStateSuccessors():
                if next_state not in explored: 
                    num += 1
                    if num % 500 == 0:
                        print(f"Checking state #{num} with move length {len(cur_state.moves)}")
                    #print("not in explored")
                    next_state.costs = copy(cur_state.costs)
                    next_state.costs.append(stateDifficulty(next_state) + moveDifficulty(cur_state, action[0], action[1]))
                    next_state.moves = copy(cur_state.moves)
                    next_state.moves.append(action)
                    #print(next_state.moves)
                    #print(frontier.heap)
                    if next_state in [tup[2] for tup in frontier.heap]:
                        frontier.update(next_state, self.getCostValue(next_state.costs))
                        #print(f"Updated {next_state} with {self.getCostValue(next_state.costs)}")
                    else:
                        frontier.push(next_state, self.getCostValue(next_state.costs))
                        #print(f"pushed {next_state} with {self.getCostValue(next_state.costs)}")


In [10]:
from get_holds import getHoldsArray
holds = getHoldsArray('../images/coolwall.jpg', [0, 40, 40], 15)
holds.sort()
print(holds)


image 1/1 /Users/liam/courses/year4/ai_cs4100/AI-Climbing/src/../images/coolwall.jpg: 1280x1184 97 holds, 397.5ms
Speed: 28.8ms preprocess, 397.5ms inference, 15.2ms postprocess per image at shape (1, 3, 1280, 1184)

0: 64x64 one 0.75, two 0.22, four 0.01, three 0.01, six 0.00, 18.5ms
Speed: 3.8ms preprocess, 18.5ms inference, 0.1ms postprocess per image at shape (1, 3, 64, 64)

0: 64x64 four 0.61, two 0.39, ten 0.00, nine 0.00, seven 0.00, 3.8ms
Speed: 0.9ms preprocess, 3.8ms inference, 0.0ms postprocess per image at shape (1, 3, 64, 64)

0: 64x64 one 0.94, two 0.06, seven 0.00, five 0.00, three 0.00, 11.7ms
Speed: 0.9ms preprocess, 11.7ms inference, 0.0ms postprocess per image at shape (1, 3, 64, 64)

0: 64x64 four 0.95, seven 0.02, nine 0.02, five 0.01, ten 0.00, 3.0ms
Speed: 1.1ms preprocess, 3.0ms inference, 0.0ms postprocess per image at shape (1, 3, 64, 64)

0: 64x64 one 0.83, four 0.14, three 0.02, downclimb 0.00, two 0.00, 3.9ms
Speed: 1.0ms preprocess, 3.9ms inference, 0.0ms

In [11]:
def moveToText(action, route):
    output = "- Move your "
    if action[0].name == LimbName.LEFT_LEG:
        output += "left leg "
    elif action[0].name == LimbName.LEFT_HAND:
        output += "left hand "
    elif action[0].name == LimbName.RIGHT_LEG:
        output += "right leg "
    elif action[0].name == LimbName.RIGHT_HAND:
        output += "right hand "
    output += "to hold number "
    output += str(route.holds.index(action[1]))
    output += " from the top"
    return output

In [12]:
route = Route(holds = holds, start1 = holds[-6], start2 = holds[-6], finish = holds[1])
lh = Limb(LimbName.LEFT_HAND, 2.5, 8, route.start_hold1)
rh = Limb(LimbName.RIGHT_HAND, 2.5, 8, route.start_hold1)
lf = Limb(LimbName.LEFT_LEG, 8, 5, route.holds[-1])
rf = Limb(LimbName.RIGHT_LEG, 8, 5, route.holds[-1])
ondra = Person(70)
liam = Person(68)
rachel = Person(66)
anna = Person(63)
short = Person(59)
state = State(lf, rf, lh, rh, short, route)
a_star = aStar(state)
results = a_star.uniformCostSearch()
for action in results:
    print(moveToText(action, route))

Checking state #500 with move length 4
Checking state #1000 with move length 7
TAAAAAAAAAAAAAAKE
[61.12128132154881, 52.65807586746385, 56.272276424560296, 37.0987672466349, 74.43977476376872, 72.16444133455786, 88.4163149617888, 74.68837596291354, 86.69350434616724, 69.5749017892546, 109.24369751484294, 107.87019325896944, 96.0281081907379]
80344.34878302015
- Move your left leg to hold number 11 from the top
- Move your right leg to hold number 11 from the top
- Move your left hand to hold number 6 from the top
- Move your right hand to hold number 6 from the top
- Move your left leg to hold number 8 from the top
- Move your right leg to hold number 8 from the top
- Move your left leg to hold number 7 from the top
- Move your right leg to hold number 7 from the top
- Move your right hand to hold number 3 from the top
- Move your left hand to hold number 3 from the top
- Move your left leg to hold number 4 from the top
- Move your right leg to hold number 4 from the top
- Move your ri

In [13]:
hi = np.array([(0, 0, 1), (0, 0, 1)])
print(hi[:, 2])

[1 1]


In [14]:
print(holds[-6])

Hold: Top left at 800.716796875, 2018.7711181640625
                Width = 77.246826171875, Height = 79.774169921875
                Difficulty = 4.72/10, Angle = 315 degrees

